In [7]:
# general
import sys, os, path
import datetime

# nn related
import torch
import torch as th
import torch.nn as nn
import numpy as np
from numpy import linalg as la

import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
import matplotlib as mpl
from matplotlib.transforms import Bbox

import sys, re, os, path
from matplotlib import axis
import seaborn as sns
colors=sns.color_palette()

# own scrips
sys.path.insert(1, '/home/leonriccius/PycharmProjects/data_driven_rans')
from scripts.preProcess import readPatchData
from scripts.preProcess import readCellCenters
from scripts.preProcess import readVectorData
from scripts.utilities import NoStdStreams
from scripts.utilities import bisection

from scripts import preProcess as pre

In [8]:
rans_path = '/home/leonriccius/Documents/Fluid_Data/rans_kaandorp/ConvDivChannel/Re7900_kOmega_100'
rans_time = max([int(entry) for entry in os.listdir(rans_path) if entry.isnumeric()])
rans_time

7000

In [9]:
with NoStdStreams():
    u = pre.readVectorData(rans_time, 'U', rans_path)
    grid = pre.readCellCenters(rans_time, rans_path)
    rs = pre.readSymTensorData(rans_time, 'turbulenceProperties:R', rans_path).reshape(-1, 3, 3)
    grad_u = pre.readTensorData(rans_time, 'grad(U)', rans_path)
    grad_k = pre.readVectorData(rans_time, 'grad(k)', rans_path)
    grad_p = pre.readVectorData(rans_time, 'grad(p)', rans_path)
    y_wall = pre.readScalarData(rans_time, 'wallDistance', rans_path)
    k = pre.readScalarData(rans_time, 'k', rans_path)
    omega = pre.readScalarData(rans_time, 'omega', rans_path)  # 'epsilon' or 'omega'
    epsilon = omega * k * 0.09  # 0.09 is beta star
    nu = 3.5714285714285714e-04

In [15]:
# define model paths
model_base = '/home/leonriccius/PycharmProjects/data_driven_rans/storage/models/kaandorp_data/weight_decay/cdc7900'
buildin_path = os.sep.join([model_base, 'builtin'])
sep_path = os.sep.join([model_base, 'sep'])

